In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.6/230.6 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.6 MB/s eta 0:00:00


In [4]:
import pandas as pd
import numpy as np
import os
import random
from tqdm.auto import tqdm
import optuna

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings(action='ignore')

DATA_PATH = '/content/drive/MyDrive/데이터 분석/데이콘 캐글 컴페티션/예슬 개인 참가/대구 교통사고 피해 예측/data/'
SEED = 42

In [5]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 3.5 MB/s eta 0:00:00


In [6]:
# Modeling & Tuning
from sklearn.model_selection import KFold
n_splits=5; seed = 42
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=seed)
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
# from ngboost import NGBRegressor
# from sklearn.svm import SVR
# from sklearn.linear_model import Ridge
# from sklearn.linear_model import Lasso
# from sklearn.linear_model import ElasticNet
# from sklearn.linear_model import LinearRegression
# from sklearn.cluster import KMeans
# from sklearn.model_selection import RandomizedSearchCV


In [7]:
train = pd.read_csv(f"{DATA_PATH}train.csv")

train_df = pd.read_csv(f"{DATA_PATH}train.csv")
test_df = pd.read_csv(f"{DATA_PATH}test.csv")
accident_df = pd.read_csv(f"{DATA_PATH}countrywide_accident.csv")
DG_cctv = pd.read_csv(f"{DATA_PATH}대구 CCTV 정보.csv", encoding='cp949')
DG_parking = pd.read_csv(f"{DATA_PATH}대구 주차장 정보.csv", encoding='cp949')
DG_children_area = pd.read_csv(f"{DATA_PATH}대구 어린이 보호 구역 정보.csv", encoding='cp949')
DG_seculight = pd.read_csv(f"{DATA_PATH}대구 보안등 정보.csv", encoding='cp949')
submission = pd.read_csv(f"{DATA_PATH}sample_submission.csv")

# KR_cctv = pd.read_csv(f'{DATA_PATH}전국무인교통단속카메라표준데이터.csv', encoding='cp949')
# KR_seculight =pd.read_csv(f'{DATA_PATH}전국보안등정보표준데이터.csv', encoding='cp949')
# KR_children_area = pd.read_csv(f'{DATA_PATH}전국어린이보호구역표준데이터.csv', encoding='cp949')
# KR_parking = pd.read_csv(f'{DATA_PATH}전국주차장정보표준데이터.csv', encoding='cp949')

# train_df.shape, test_df.shape

# DATA_PATH = '/kaggle/input/daegu-dataset-2/'

# train = pd.read_csv(f"{DATA_PATH}train.csv")

# train_df = pd.read_csv(f"{DATA_PATH}train.csv")
# test_df = pd.read_csv(f"{DATA_PATH}test.csv")
# accident_df = pd.read_csv(f"{DATA_PATH}countrywide_accident.csv")
# DG_cctv = pd.read_csv(f"{DATA_PATH}DG_cctv.csv", encoding='cp949')
# DG_parking = pd.read_csv(f"{DATA_PATH}DG_parking.csv", encoding='cp949')
# DG_children_area = pd.read_csv(f"{DATA_PATH}DG_children_area.csv", encoding='cp949')
# DG_seculight = pd.read_csv(f"{DATA_PATH}DG_seculight.csv", encoding='cp949')
# submission = pd.read_csv(f"{DATA_PATH}sample_submission.csv")

# DATA_PATH = '/kaggle/input/daegu-kr-dataset/'

# KR_cctv = pd.read_csv(f'{DATA_PATH}KR_cctv.csv', encoding='cp949')
# KR_seculight =pd.read_csv(f'{DATA_PATH}KR_seculight.csv', encoding='cp949')
# KR_children_area = pd.read_csv(f'{DATA_PATH}KR_children_area.csv', encoding='cp949')
# KR_parking = pd.read_csv(f'{DATA_PATH}KR_parking.csv', encoding='cp949')

train_df.shape, test_df.shape

((39609, 23), (10963, 8))

In [8]:
# 안개 삭제
train_df.drop(train_df[train_df['기상상태'] == '안개'].index, inplace=True)
accident_df.drop(accident_df[accident_df['기상상태'] == '안개'].index, inplace=True)

# 해빙, nan 삭제
accident_df.drop(accident_df[accident_df['노면상태'] == '해빙'].index, inplace=True)
accident_df.drop(accident_df[accident_df['노면상태'].isnull()].index, inplace=True)

# 철길건널목 삭제
accident_df.drop(accident_df[accident_df['사고유형'] == '철길건널목'].index, inplace=True)

In [9]:
train_df = pd.concat([train, accident_df], axis=0).reset_index().drop(columns='index')

In [10]:
location_pattern = r'(\S+) (\S+) (\S+)'

train_df[['도시', '구', '동']] = train_df['시군구'].str.extract(location_pattern)
train_df = train_df.drop(columns=['시군구'])

# test 데이터 반영
test_df[['도시', '구', '동']] = test_df['시군구'].str.extract(location_pattern)
test_df = test_df.drop(columns=['시군구'])

In [11]:
road_pattern = r'(.+) - (.+)'

train_df[['도로형태1', '도로형태2']] = train_df['도로형태'].str.extract(road_pattern)
train_df = train_df.drop(columns=['도로형태'])

# test 데이터 반영
test_df[['도로형태1', '도로형태2']] = test_df['도로형태'].str.extract(road_pattern)
test_df = test_df.drop(columns=['도로형태'])

In [12]:
gwangyuk = ['대구광역시','인천광역시','광주광역시','부산광역시','울산광역시','대전광역시']

mask = train_df['도시'].isin(gwangyuk)
train_df = train_df[mask].reset_index().drop(columns='index')

In [ ]:
train_df.columns.tolist()

['ID',
 '사고일시',
 '요일',
 '기상상태',
 '노면상태',
 '사고유형',
 '사고유형 - 세부분류',
 '법규위반',
 '가해운전자 차종',
 '가해운전자 성별',
 '가해운전자 연령',
 '가해운전자 상해정도',
 '피해운전자 차종',
 '피해운전자 성별',
 '피해운전자 연령',
 '피해운전자 상해정도',
 '사망자수',
 '중상자수',
 '경상자수',
 '부상자수',
 'ECLO',
 '도시',
 '구',
 '동',
 '도로형태1',
 '도로형태2']

# Feature 실험

### 시간추가

In [ ]:
import holidays
kr_holidays = holidays.KR()
# 시간 관련
train_df['date'] = pd.to_datetime(train_df['사고일시'])
train_df['연'] = train_df['date'].dt.year.astype(int)
train_df['월'] = train_df['date'].dt.month.astype(int)
train_df['일'] = train_df['date'].dt.day.astype(int)
train_df['시간'] = train_df['date'].dt.hour.astype(int)
train_df['요일'] = train_df['date'].dt.day_of_week.astype(int)
train_df['공휴일'] = train_df['date'].apply(lambda x : int(x in kr_holidays)).astype(int)

test_df['date'] = pd.to_datetime(test_df['사고일시'])
test_df['연'] = test_df['date'].dt.year.astype(int)
test_df['월'] = test_df['date'].dt.month.astype(int)
test_df['일'] = test_df['date'].dt.day.astype(int)
test_df['시간'] = test_df['date'].dt.hour.astype(int)
test_df['요일'] = test_df['date'].dt.day_of_week.astype(int)
test_df['공휴일'] = test_df['date'].apply(lambda x : int(x in kr_holidays)).astype(int)

train_df = train_df.drop(columns=['사고일시','date'])
test_df = test_df.drop(columns=['사고일시','date'])

### 총부상자

In [ ]:
train_df['월동_총부상자'] = train_df['사망자수'] + train_df['중상자수'] + train_df['경상자수'] + train_df['부상자수']

casualty = train_df.groupby(['월','도시','구','동'])[['월동_총부상자']].mean().reset_index()
train_df = train_df.drop(columns = '월동_총부상자')

train_df = pd.merge(train_df,casualty, on = ['월','도시','구','동'], how='left')
test_df = pd.merge(test_df,casualty, on = ['월','도시','구','동'], how='left')



### eclo mean

In [ ]:
tmp = train_df.groupby(['시간','요일'])[['ECLO']].mean().reset_index()
tmp.columns = ['시간','요일','ECLO_mean']

train_df = train_df.merge(tmp, on= ['시간','요일'], how='left')
test_df = test_df.merge(tmp, on= ['시간','요일'], how='left')

### train only

In [ ]:
# train_ft1 = pd.concat([train,accident_df], axis= 0).reset_index()

train_ft1 = train

In [ ]:
# 시간
train_ft1['date'] = pd.to_datetime(train_ft1['사고일시'])
train_ft1['연'] = train_ft1['date'].dt.year.astype(int)
train_ft1['월'] = train_ft1['date'].dt.month.astype(int)
train_ft1['일'] = train_ft1['date'].dt.day.astype(int)
train_ft1['시간'] = train_ft1['date'].dt.hour.astype(int)
train_ft1['요일'] = train_ft1['date'].dt.day_of_week.astype(int)

# 시군구
location_pattern = r'(\S+) (\S+) (\S+)'
train_ft1[['도시', '구', '동']] = train_ft1['시군구'].str.extract(location_pattern)
train_ft1 = train_ft1.drop(columns=['시군구'])

# 도로형태

road_pattern = r'(.+) - (.+)'
train_ft1[['도로형태1', '도로형태2']] = train_ft1['도로형태'].str.extract(road_pattern)
train_ft1 = train_ft1.drop(columns=['도로형태'])

In [ ]:
def categorize_time_period(hour):
    if 6 <= hour < 9:  # 아침 출퇴근 시간
        return 0
    elif 9 <= hour < 12:  # 아침
        return 1
    elif 12 <= hour < 14:  # 점심
        return 2
    elif 14 <= hour < 18:  # 오후
        return 3
    elif 18 <= hour < 21:  # 저녁
        return 4
    elif 21 <= hour < 24:  # 밤
        return 5
    else:  # 새벽
        return 6

train_ft1['시간대'] = train_ft1['시간'].apply(categorize_time_period)

train_df['시간대'] = train_df['시간'].apply(categorize_time_period)
test_df['시간대'] = test_df['시간'].apply(categorize_time_period)

In [ ]:

df = train_ft1.groupby(['가해운전자 차종'])[['ECLO']].mean().reset_index()
df0 = train_ft1.groupby(['가해운전자 차종'])[['ECLO']].count().reset_index().iloc[:,1:]
df1 = train_ft1.groupby(['가해운전자 차종'])[['ECLO']].sum().reset_index().iloc[:,1:]
df2 = train_ft1.groupby(['가해운전자 차종'])[['ECLO']].min().reset_index().iloc[:,1:]
df3 = train_ft1.groupby(['가해운전자 차종'])[['ECLO']].max().reset_index().iloc[:,1:]
df4 = train_ft1.groupby(['가해운전자 차종'])[['ECLO']].median().reset_index().iloc[:,1:]
df5 = train_ft1.groupby(['가해운전자 차종'])[['ECLO']].std().reset_index().iloc[:,1:]


df_acc = pd.concat([df, df4,df0, df1, df2, df3, df5], axis=1)
df_acc.columns = ['가해운전자 차종','median','mean','count','sum','min','max','std']
# df_acc['max_mean'] = df_acc['max'] * df_acc['mean']
# df_acc['max_median'] = df_acc['max'] * df_acc['median']
# df_acc['max_std'] = df_acc['max'] * df_acc['std']
# df_acc['mean_median'] = df_acc['mean'] * df_acc['median']
# df_acc['max_count'] = df_acc['max'] * (df_acc['count'])
# df_acc['max_count_medianmean'] = df_acc['max'] * df_acc['count'] * (df_acc['mean'] + df_acc['median'])/2
# df_acc['max_count_log'] = df_acc['max'] * np.log(df_acc['count'])
# df_acc['max_count_log_mean_log'] = df_acc['max'] * np.log(df_acc['count']) *np.log(df_acc['mean'])
# df_acc['count_log_median'] = np.log(df_acc['count']) * df_acc['median']
# df_acc['max_count_log_median'] = df_acc['max'] * np.log(df_acc['count']) *df_acc['median']
# df_acc['max_count_log_mean'] = df_acc['max'] * np.log(df_acc['count']) *df_acc['mean']
df_acc['max_std_count_log'] = df_acc['max'] * np.log(df_acc['count']) *df_acc['std']


# max count_log의 조합을 기준으로 정렬해서 라벨 인코딩 (비슷한 범주가 라벨링된다는 가정하에 범주를 평균할 수 있으므로)
df_acc = df_acc.sort_values(by='max_std_count_log', ascending=False)
# display(df_acc)

df_cols = df_acc['가해운전자 차종'].tolist()

label_mapping = {label: index for index, label in enumerate(df_cols)}
train_ft1['가해운전자 차종_sorted'] = train_ft1['가해운전자 차종'].map(label_mapping).astype(int)

# for col in train_ft1[['가해운전자 차종_sorted', '가해운전자 차종']]:
#     sns.scatterplot(x=train_ft1[col], y=train_ft1['ECLO'])
#     break

In [ ]:
df = train_ft1.groupby(['구','시간대','요일','사고유형','도로형태2'])[['가해운전자 차종_sorted']].mean().reset_index()
df

,구,시간대,요일,사고유형,도로형태2,가해운전자 차종_sorted
0,남구,0,0,차대사람,교차로부근,1.000000
1,남구,0,0,차대사람,교차로횡단보도내,1.000000
2,남구,0,0,차대사람,기타,3.200000
3,남구,0,0,차대사람,주차장,2.000000
4,남구,0,0,차대차,교차로부근,2.833333
...,...,...,...,...,...,...
3775,중구,6,6,차대차,교량,2.000000
3776,중구,6,6,차대차,교차로부근,2.000000
3777,중구,6,6,차대차,교차로안,2.142857
3778,중구,6,6,차대차,기타,2.304348


In [ ]:
train_df = pd.merge(train_df,df, on = ['구','시간대','요일','사고유형','도로형태2'], how='left')
test_df = pd.merge(test_df,df, on = ['구','시간대','요일','사고유형','도로형태2'], how='left')


In [ ]:

df = train_ft1.groupby(['피해운전자 상해정도'])[['ECLO']].mean().reset_index()
df1 = train_ft1.groupby(['피해운전자 상해정도'])[['ECLO']].min().reset_index().iloc[:,1:]

df_acc = pd.concat([df,df1], axis=1)
df_acc.columns = ['피해운전자 상해정도','mean','min']
df_acc = df_acc.sort_values(by='min', ascending=False)

df_cols = df_acc['피해운전자 상해정도'].tolist()

label_mapping = {label: index for index, label in enumerate(df_cols)}
train_ft1['피해운전자 상해정도_sorted'] = train_ft1['피해운전자 상해정도'].map(label_mapping)

# for col in train_ft1[['피해운전자 상해정도_sorted', '피해운전자 상해정도']]:
#     sns.scatterplot(x=train_ft1[col], y=train_ft1['ECLO'])
#     break

In [ ]:
df = train_ft1.groupby(['구','시간대','요일','사고유형','도로형태2'])[['피해운전자 상해정도_sorted']].mean().reset_index()
df

,구,시간대,요일,사고유형,도로형태2,피해운전자 상해정도_sorted
0,남구,0,0,차대사람,교차로부근,2.000000
1,남구,0,0,차대사람,교차로횡단보도내,1.000000
2,남구,0,0,차대사람,기타,3.200000
3,남구,0,0,차대사람,주차장,2.000000
4,남구,0,0,차대차,교차로부근,2.500000
...,...,...,...,...,...,...
3775,중구,6,6,차대차,교량,2.000000
3776,중구,6,6,차대차,교차로부근,2.285714
3777,중구,6,6,차대차,교차로안,2.000000
3778,중구,6,6,차대차,기타,2.391304


In [ ]:
train_df = pd.merge(train_df,df, on = ['구','시간대','요일','사고유형','도로형태2'], how='left')
test_df = pd.merge(test_df,df, on = ['구','시간대','요일','사고유형','도로형태2'], how='left')


In [ ]:
train_df = train_df.drop(columns = '시간대')
test_df = test_df.drop(columns = '시간대')

# 인코딩, 스케일링

In [ ]:
train_df = train_df.fillna(0)
test_df = test_df.fillna(0)

In [ ]:
train_df.columns.tolist()

['ID',
 '요일',
 '기상상태',
 '노면상태',
 '사고유형',
 '사고유형 - 세부분류',
 '법규위반',
 '가해운전자 차종',
 '가해운전자 성별',
 '가해운전자 연령',
 '가해운전자 상해정도',
 '피해운전자 차종',
 '피해운전자 성별',
 '피해운전자 연령',
 '피해운전자 상해정도',
 '사망자수',
 '중상자수',
 '경상자수',
 '부상자수',
 'ECLO',
 '도시',
 '구',
 '동',
 '도로형태1',
 '도로형태2',
 '연',
 '월',
 '일',
 '시간',
 '공휴일',
 '월동_총부상자',
 'ECLO_mean',
 '가해운전자 차종_sorted',
 '피해운전자 상해정도_sorted']

In [ ]:
test_x = test_df.drop(columns = ['ID','도시'])
train_x = train_df[test_x.columns]

train_y = train_df['ECLO']

In [ ]:
num_cols = train_x.select_dtypes(exclude='object').columns.tolist()
num_cols

['요일',
 '연',
 '월',
 '일',
 '시간',
 '공휴일',
 '월동_총부상자',
 'ECLO_mean',
 '가해운전자 차종_sorted',
 '피해운전자 상해정도_sorted']

In [ ]:
cat_cols = train_x.select_dtypes(include='object').columns.tolist()
cat_cols

['기상상태', '노면상태', '사고유형', '구', '동', '도로형태1', '도로형태2']

In [ ]:
cat_cols
# 시간, 공간 정보 이외 모두 one hot encoding

cat_cols1 = ['요일','구','동']
cat_cols2 = [col for col in cat_cols if col not in cat_cols1]
cat_cols2

['기상상태', '노면상태', '사고유형', '도로형태1', '도로형태2']

In [ ]:
!pip install category_encoders

In [ ]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, OrdinalEncoder
import category_encoders as ce

one_hot = OneHotEncoder()
label = LabelEncoder()
ordinal = OrdinalEncoder()
target_enc = ce.TargetEncoder()

In [ ]:
train_x[cat_cols1] = train_x[cat_cols1].astype(str)

In [ ]:
# cols1 라벨인코딩
for col in cat_cols1 :
    train_x[col] = target_enc.fit_transform(train_x[col],train_y)
    test_x[col] = target_enc.transform(test_x[col])

In [ ]:
train_x[cat_cols2] = train_x[cat_cols2].astype(str)

In [ ]:
# cols 2 원핫인코딩
tmp = pd.DataFrame(one_hot.fit_transform(train_x[cat_cols2]).toarray())
tmp.columns = one_hot.get_feature_names_out()
pd.DataFrame(tmp)

train_x = pd.concat([train_x,tmp],axis=1).drop(columns= cat_cols2)

tmp = pd.DataFrame(one_hot.fit_transform(test_x[cat_cols2]).toarray())
tmp.columns = one_hot.get_feature_names_out()
pd.DataFrame(tmp)

test_x = pd.concat([test_x,tmp],axis=1).drop(columns= cat_cols2)

train_x.shape, test_x.shape

((158959, 45), (10963, 41))

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
# scaler = StandardScaler()

# train_df[num_cols] = scaler.fit_transform(train_df[num_cols])
# test_df[num_cols] = scaler.transform(test_df[num_cols])

scaler = StandardScaler()

train_x[num_cols] = scaler.fit_transform(train_x[num_cols])
test_x[num_cols] = scaler.transform(test_x[num_cols])

# 검증 및 평가

In [ ]:
target = train_y

In [ ]:
from sklearn.metrics import make_scorer

def rmsle(y_actual, y_pred):
    diff = np.log1p(y_pred) - np.log1p(y_actual)
    mean_error = np.mean(np.square(diff))

    return np.sqrt(mean_error)

rmsle_score = make_scorer(rmsle, greater_is_better=False)

In [ ]:
# import optuna
# from ngboost import NGBRegressor
# from ngboost.distns import LogNormal

# # Assume train_x, target, rmsle_score are defined

# def ngboost_objective(trial):
#     params = {
#         'Dist': LogNormal,  # You may choose a different distribution
#         # 'Base': trial.suggest_categorical('Base', ['DecisionTree', 'RandomForest']),  # Base learner
#         'n_estimators': trial.suggest_int('n_estimators', 100, 5000),
#         'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.1, log=True),
#         'minibatch_frac': trial.suggest_float('minibatch_frac', 0.1, 1.0),
#         'col_sample': trial.suggest_float('col_sample', 0.1, 1.0),
#         'verbose': 0  # Adjust verbosity as needed
#     }

#     cv = KFold(n_splits=5, shuffle=True, random_state=SEED)
#     model = NGBRegressor(**params, random_state=SEED)
#     scores = cross_val_score(model, train_x, target, cv=cv, scoring=rmsle_score, n_jobs=-1)

#     return scores.mean()

In [ ]:
def xgb_objective(trial):
    params = {
        'objective': 'reg:squaredlogerror',
        'eval_metric': 'rmsle',
        'n_estimators': trial.suggest_int('n_estimators', 100, 5000),
        'max_depth': trial.suggest_int('max_depth', 1, 25),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.1, log=True),
        'gamma': trial.suggest_float('gamma', 0, 1),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 20),
        'subsample': trial.suggest_float('subsample', 0.05, 1),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.05, 1),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 10),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 10),
    }

    cv = KFold(n_splits=5, shuffle=True, random_state=SEED)
    model = XGBRegressor(**params, random_state=SEED)
    scores = cross_val_score(model, train_x, target, cv=cv, scoring=rmsle_score, n_jobs=-1)

    return scores.mean()

In [ ]:
import optuna

sampler = optuna.samplers.TPESampler(seed=SEED)
study = optuna.create_study(direction='maximize', sampler=sampler)
study.optimize(xgb_objective, n_trials=100)

[I 2023-12-02 08:45:47,136] A new study created in memory with name: no-name-758ad9e2-f36a-4bfb-bd24-93973c106a5e
[I 2023-12-02 08:47:56,952] Trial 0 finished with value: -0.4216957502233484 and parameters: {'n_estimators': 1935, 'max_depth': 24, 'learning_rate': 0.029106359131330698, 'gamma': 0.5986584841970366, 'min_child_weight': 4, 'subsample': 0.19819479431939252, 'colsample_bytree': 0.10517943155978948, 'reg_alpha': 8.661761457749352, 'reg_lambda': 6.011150117432088}. Best is trial 0 with value: -0.4216957502233484.
[I 2023-12-02 08:51:51,545] Trial 1 finished with value: -0.4213641123284787 and parameters: {'n_estimators': 3570, 'max_depth': 1, 'learning_rate': 0.08706020878304858, 'gamma': 0.8324426408004217, 'min_child_weight': 5, 'subsample': 0.2227337188467456, 'colsample_bytree': 0.22423428436076215, 'reg_alpha': 3.0424224295953772, 'reg_lambda': 5.247564316322379}. Best is trial 1 with value: -0.4213641123284787.
[I 2023-12-02 08:54:46,984] Trial 2 finished with value: -0.

In [ ]:
# -0.4180133168978298

xgb_best_params = {'n_estimators': 3241, 'max_depth': 24, 'learning_rate': 0.024010747652112696, 'gamma': 0.7071727358928532, 'min_child_weight': 2, 'subsample': 0.8062272924485435, 'colsample_bytree': 0.8429061023478192, 'reg_alpha': 0.1812908822349264, 'reg_lambda': 1.7344690669541394}


## 시드앙상블

In [ ]:
seed_scores = []

for SEED in tqdm(range(1000)) :
    cv = KFold(n_splits=5, shuffle=True, random_state=SEED)
    scores = []
    # models = []
    for tri , vai in cv.split(train_x): # k fold 앙상블
        # 학습 데이터
        x_train = train_x.iloc[tri]
        y_train = train_y.iloc[tri]

        # 검증 데이터
        x_valid = train_x.iloc[vai]
        y_valid = train_y.iloc[vai]

        model = XGBRegressor(random_state=SEED, n_jobs=-1, verbosity = 0)
        model.fit(x_train, y_train)
        pred = model.predict(x_valid)
        score = rmsle(y_valid,pred)
        scores.append(score)

    avg_score = np.mean(scores)
    print(f"SEED {SEED} - RMSLE : {avg_score}")
    seed_scores.append(avg_score)


  0%|          | 0/1000 [00:00<?, ?it/s]

SEED 0 - RMSLE : 0.4355168989540685
SEED 1 - RMSLE : 0.43563630459865543
SEED 2 - RMSLE : 0.43587797814448404
SEED 3 - RMSLE : 0.4356939471790092
SEED 4 - RMSLE : 0.4358951164853999
SEED 5 - RMSLE : 0.4354450363150996
SEED 6 - RMSLE : 0.43608519673566287
SEED 7 - RMSLE : 0.4358704094739
SEED 8 - RMSLE : 0.43578137029562825
SEED 9 - RMSLE : 0.43525937148032073
SEED 10 - RMSLE : 0.435940410636489
SEED 11 - RMSLE : 0.43568737764485743
SEED 12 - RMSLE : 0.43500664057429106
SEED 13 - RMSLE : 0.43591652203307013
SEED 14 - RMSLE : 0.4357967656395824
SEED 15 - RMSLE : 0.4352504419720913
SEED 16 - RMSLE : 0.4357145292666519
SEED 17 - RMSLE : 0.4353609950953806
SEED 18 - RMSLE : 0.43569243994102447
SEED 19 - RMSLE : 0.43562173728811715
SEED 20 - RMSLE : 0.43574646395021305
SEED 21 - RMSLE : 0.43583642463117755
SEED 22 - RMSLE : 0.435430965216165
SEED 23 - RMSLE : 0.43575813091426746
SEED 24 - RMSLE : 0.43569709863215805
SEED 25 - RMSLE : 0.435789981863923
SEED 26 - RMSLE : 0.43549670773605165
SE

In [ ]:
seed_ord = []

for idx, score in enumerate(sorted(seed_scores)):
    seed_ord.append(seed_scores.index(score))

print(seed_ord)